<div id="container" style="position:relative;">
<div style="float:left"><h1> Mahyar Sabouniaghdam</h1></div>
<div style="position:relative; float:right"><img style="height:65px" src ="https://drive.google.com/uc?export=view&id=1EnB0x-fdqMp6I5iMoEBBEuxB_s7AmE2k" />
</div>
</div>

#### Gmail: saboonimahyar@gmail.com

#### LinkedIn Profile: https://www.linkedin.com/in/mahyar-sabouniaghdam/


# BrainStation Capstone Project : Stack Overflow Questions Quality Rating

## Table of Contents:

* [Artificial Neural Networks (ANNs)](#1)
* [Multi Class Neural Networks](#2)
* [Random Forest ](#3)
* [XGBoost ](#4)

Note: This notebook was run in the deeplearning environment.

This notebook in a supplemantry notebook which we will run some deeplearning and rerun my advanced models again:

- Let's import some packages:

In [7]:
# Importing packages

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [15]:
from scipy import stats 

from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, 
    roc_auc_score, roc_curve, auc,
    plot_confusion_matrix, plot_roc_curve
)

from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

Let's read our data:

In [8]:
X_train = pd.read_csv('X_remainder_vect_tfidf.csv')
X_test = pd.read_csv('X_test_vect_tfidf.csv')
y_train = pd.read_csv('y_remainder.csv')
y_test = pd.read_csv('y_test.csv')

In [3]:
# check dataframes shapes

print(f"The shape of the X_train dataframe is: {X_train.shape}.")
print(f"The shape of the X_test dataframe is: {X_test.shape}.\n")
print(f"The shape of the y_train dataframe is: {y_train.shape}.")
print(f"The shape of the y_test dataframe is: {y_test.shape}.\n")

The shape of the X_train dataframe is: (22498, 2018).
The shape of the X_test dataframe is: (7500, 2018).

The shape of the y_train dataframe is: (22498, 1).
The shape of the y_test dataframe is: (7500, 1).



### Scaling Data

Now I am going to apply MinMaxScaler to our dataset:

In [10]:
from sklearn.preprocessing import MinMaxScaler

# instantiate the model
scaler = MinMaxScaler()

# fit the model
scaler = scaler.fit(X_train)

# transform
X_train_scaled= scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

- Now let's create some functions for evaluation of ANN and printing the score: 

In [5]:
def evaluate_nn(true, pred, train=True):
    if train:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")
        
    elif train==False:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")
        
def plot_learning_evolution(r):
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    plt.plot(r.history['loss'], label='Loss')
    plt.plot(r.history['val_loss'], label='val_Loss')
    plt.title('Loss evolution during trainig')
    plt.legend()

    plt.subplot(2, 2, 2)
    plt.plot(r.history['AUC'], label='AUC')
    plt.plot(r.history['val_AUC'], label='val_AUC')
    plt.title('AUC score evolution during trainig')
    plt.legend();

def nn_model(num_columns, num_labels, hidden_units, dropout_rates, learning_rate):
    inp = tf.keras.layers.Input(shape=(num_columns, ))
    x = BatchNormalization()(inp)
    x = Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = Dense(hidden_units[i], activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(dropout_rates[i + 1])(x)
    x = Dense(num_labels, activation='sigmoid')(x)
  
    model = Model(inputs=inp, outputs=x)
    model.compile(optimizer=Adam(learning_rate), loss='binary_crossentropy', metrics=[AUC(name='AUC')])
    return model

In [6]:
def print_score(true, pred, train=True):
    if train:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")
        
    elif train==False:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")

## ANN Model <a id="1"></a>

In [15]:
num_columns = X_train_scaled.shape[1]
num_labels = 1
hidden_units = [150, 150, 150]
dropout_rates = [0.1, 0, 0.1, 0]
learning_rate = 1e-3


model = nn_model(
    num_columns=num_columns, 
    num_labels=num_labels,
    hidden_units=hidden_units,
    dropout_rates=dropout_rates,
    learning_rate=learning_rate
)
r = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=20,
    batch_size=32
)

Epoch 1/20
704/704 [==============================] - 5s 7ms/step - loss: 0.5935 - AUC: 0.7526 - val_loss: 0.5535 - val_AUC: 0.8069
Epoch 2/20
704/704 [==============================] - 4s 6ms/step - loss: 0.4293 - AUC: 0.8803 - val_loss: 0.5791 - val_AUC: 0.8105
Epoch 3/20
704/704 [==============================] - 4s 6ms/step - loss: 0.3686 - AUC: 0.9137 - val_loss: 0.6338 - val_AUC: 0.8085
Epoch 4/20
704/704 [==============================] - 4s 6ms/step - loss: 0.3202 - AUC: 0.9359 - val_loss: 0.8116 - val_AUC: 0.7916
Epoch 5/20
704/704 [==============================] - 5s 6ms/step - loss: 0.2811 - AUC: 0.9509 - val_loss: 0.9192 - val_AUC: 0.7804
Epoch 6/20
704/704 [==============================] - 5s 6ms/step - loss: 0.2457 - AUC: 0.9626 - val_loss: 1.0817 - val_AUC: 0.7732
Epoch 7/20
704/704 [==============================] - 4s 6ms/step - loss: 0.2128 - AUC: 0.9720 - val_loss: 1.1727 - val_AUC: 0.7631
Epoch 8/20
704/704 [==============================] - 5s 6ms/step - loss: 0.

In [16]:
y_test_pred = model.predict(X_test_scaled)
evaluate_nn(y_test, y_test_pred.round(), train=False)

Test Result:
Accuracy Score: 70.44%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.648215     0.756625    0.7044     0.702420      0.709098
recall        0.712287     0.698243    0.7044     0.705265      0.704400
f1-score      0.678742     0.726263    0.7044     0.702502      0.705430
support    3288.000000  4212.000000    0.7044  7500.000000   7500.000000
_______________________________________________
Confusion Matrix: 
 [[2342  946]
 [1271 2941]]



- The Test accuracy score for ANN is 70.5% which is lower from our XGBoost model in previous notebook.

## Multi Class Neural Networks <a id="2"></a>

- Let's import packages:

In [18]:
# Import packages

from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [44]:
# random seeds for reproducibility
tf.random.set_seed(123)

# Create a new sequential model
model = keras.Sequential()

# Declare the hidden layers
model.add(layers.Dense(40, activation="relu"))
model.add(layers.Dense(40, activation="relu"))
model.add(layers.Dense(40, activation="relu"))

# Declare the output layer
model.add(layers.Dense(3, activation="softmax"))

# Compile the model
model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # Metric used to evaluate model
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

In [45]:
# Fit the model

history = model.fit(X_train, y_train, epochs=50, verbose=0)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [46]:
# Evaluate the model using the test data and generate predictions

train_accuracy = history.history["sparse_categorical_accuracy"][-1]
result = model.evaluate(X_test, y_test, verbose=0)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {result[1]:.4f}")

predictions = model.predict(X_test)

Train Accuracy: 0.8358
Test Accuracy: 0.7936


- Here we have 79% Test score which is almost the same as our XGBoost Model in previos notebook.

I will rerun these 2 models again here for comparing with NNs models:

## Random Forest <a id="3"></a>

- Let's run the Random Forest model here again:

In [22]:
rf_clf = RandomForestClassifier(n_estimators=100)

rf_clf.fit(X_train, y_train)

y_train_pred = rf_clf.predict(X_train_scaled)
y_test_pred = rf_clf.predict(X_test_scaled)

print_score(y_train, y_train_pred, train=True)
print_score(y_test, y_test_pred, train=False)

C:\Users\mahya\AppData\Local\Temp\ipykernel_5464\892756727.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X_train, y_train)


Train Result:
Accuracy Score: 81.99%
_______________________________________________
CLASSIFICATION REPORT:
                     0             1  accuracy     macro avg  weighted avg
precision     0.812752      0.825046  0.819895      0.818899      0.819610
recall        0.770105      0.859363  0.819895      0.814734      0.819895
f1-score      0.790854      0.841855  0.819895      0.816354      0.819303
support    9948.000000  12550.000000  0.819895  22498.000000  22498.000000
_______________________________________________
Confusion Matrix: 
 [[ 7661  2287]
 [ 1765 10785]]

Test Result:
Accuracy Score: 71.60%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.688477     0.735095     0.716     0.711786      0.714657
recall        0.643248     0.772792     0.716     0.708020      0.716000
f1-score      0.665094     0.753472     0.716     0.709283      0.714727
support    3288.00

- The Test accuracy score for Random Forest Classifier is 71.6% which is lower from our XGBoost model in previous notebook

## XGBoost  <a id="4"></a>

In [10]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(use_label_encoder=False)

xgb_clf.fit(X_train_scaled, y_train)

y_test_pred = xgb_clf.predict(X_test_scaled)

print_score(y_test, y_test_pred, train=False)

C:\Users\mahya\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Test Result:
Accuracy Score: 79.43%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1  accuracy    macro avg  weighted avg
precision     0.770375     0.812310  0.794267     0.791342      0.793926
recall        0.756083     0.824074  0.794267     0.790078      0.794267
f1-score      0.763162     0.818150  0.794267     0.790656      0.794043
support    3288.000000  4212.000000  0.794267  7500.000000   7500.000000
_______________________________________________
Confusion Matrix: 
 [[2486  802]
 [ 741 3471]]



- As we can see here the Test Accuracy score for XGBoost model is 79.43% and it is better han Multi Class NNs model 79.36%. The score is almost the same as XGBoost model in the previous notebook.

# Conclusion:

- It is ture that my NNs models have lower test scores than my XGBoost model in this notebook. NNs models are more complex and I tried and wanted to run some other NNs models but because of computational limits I couldnot. So, my final model would be XGBoost and I will continue with the one I evaluated on previous notebook.

- Now we can go back to the previous notebook for final insights, final model and next steps part.